In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import cv2
import os


In [44]:
# Set the paths to your dataset
train_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
val_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

# Image dimensions and batch size

In [38]:
def load_dataset(base_path):
    categories = ['PNEUMONIA', 'NORMAL']
    data = []
    labels = []

    for label, category in enumerate(categories):
        folder_path = os.path.join(base_path, category)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            image = load_and_preprocess_image(img_path)
            data.append(image)
            labels.append(label)

    return np.array(data), np.array(labels)

In [39]:
train_data, train_labels = load_dataset(train_path)
test_data, test_lables = load_dataset(test_path)
val_data, val_lables = load_dataset(val_path)

In [40]:
image_size = (224, 224)
batch_size = 32

In [47]:
#Tang cuong du lieu trainning data augmentation
train_data_gen = ImageDataGenerator(
    rescale = 1/255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest' 
)

In [49]:
# preprocess and augment trainning data
train_data = train_data_gen.flow_from_directory(
    directory = train_path,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = "binary"
)

Found 5216 images belonging to 2 classes.


In [50]:
#preprocess test data set
test_data_gen = ImageDataGenerator(rescale = 1/255)
test_data = test_data_gen.flow_from_directory(
    directory = test_path,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = "binary"
)

Found 624 images belonging to 2 classes.


In [51]:
val_data_gen = ImageDataGenerator(rescale = 1/255)
val_data = test_data_gen.flow_from_directory(
    directory = val_path,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = "binary"
)

Found 16 images belonging to 2 classes.


In [65]:
# Sử dụng VGG thuộc CNN
#VGG16(include_top=False): Bỏ các tầng Fully Connected gốc của VGG16.
#weights='imagenet': Dùng trọng số đã được huấn luyện trên tập ImageNet.
base_model = VGG16(include_top=False, weights = "imagenet", input_shape= ((224,224,3)))


#Không cập nhật trọng số (weights) của tầng đó trong quá trình huấn luyện.
#Giúp giảm số lượng tham số cần huấn luyện, tránh overfitting khi dữ liệu nhỏ.
for layer in base_model.layers: #duyệt qua từng tầng của mô hình
    layer.trainable = False 


#fully connected để phân loại
x = layers.Flatten()(base_model.output) # biến các feature thành 1D

#Giữ nguyên các giá trị dương, đặt 0 cho các giá trị âm
x = layers.Dense(512, activation = "relu")(x) # 512 neuron, relu là mô hình phi tuyến tính giúp học các đặc trưng phức tạp

x =layers.Dropout(0.5)(x) # Loại bỏ ngẫu nhiên 50% số neurons giúp giảm overfitting

#Chuyển đổi đầu ra thành giá trị trong khoảng (0,1) → biến nó thành xác suất
x = layers.Dense(1, activation='sigmoid')(x) # 1 neuron

model = Model(base_model.input, x)

In [77]:
# Tối ưu tham số
learning_rate = 0.001
batch_size = 64
epochs = 10
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
             loss = "binary_crossentropy",
             metrics = ["accuracy"])


In [78]:
# callbacks

#Giảm learning rate nếu mô hình không còn cải thiện để giúp tiếp tục tối ưu.
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

#Dừng huấn luyện sớm nếu val_loss không cải thiện trong một số epochs liên tiếp.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [79]:
history = model.fit(train_data,
                    batch_size=batch_size,
                    epochs=epochs,
                    validation_data=val_data,
                    callbacks=[lr_scheduler, early_stopping])

Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 92s 527ms/step - accuracy: 0.8983 - loss: 0.2388 - val_accuracy: 0.7500 - val_loss: 0.3803 - learning_rate: 0.0010
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 89s 522ms/step - accuracy: 0.9184 - loss: 0.1988 - val_accuracy: 0.9375 - val_loss: 0.1871 - learning_rate: 0.0010
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 90s 530ms/step - accuracy: 0.9192 - loss: 0.1848 - val_accuracy: 0.7500 - val_loss: 0.6753 - learning_rate: 0.0010
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 88s 518ms/step - accuracy: 0.9293 - loss: 0.1813 - val_accuracy: 0.7500 - val_loss: 0.5011 - learning_rate: 0.0010
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 89s 523ms/step - accuracy: 0.9303 - loss: 0.1807 - val_accuracy: 0.8125 - val_loss: 0.2916 - learning_rate: 0.0010
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 90s 525ms/step - accuracy: 0.9373 - loss: 0.1598 - val_accuracy: 0.8125 - val_loss: 0.3109 - learning_rate: 5.0000e-04
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 89s 523ms/step - accuracy:

In [103]:
# Save the model
model.save('/kaggle/working/cnn_model_chest-xray-pneumonia.h5')